# Exercise 4.8 Grouping Data and Aggregating Variables

## Importing Libraries

In [1]:
import numpy as np 
import pandas as pd
import os

## Importing Data

In [2]:
# Set path for data 
path = r'/Users/brookecoffey/10-07-2023 Instacart Basket Analysis'

In [7]:
# Import ords_prods_merged dataset
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merged_updated.pkl'))

## 02/03 In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [8]:
# Create a subset for comparison 
df = ords_prods_merge[:1000000]

In [9]:
df.shape

(1000000, 21)

In [10]:
# Group whole dataframe by department_id and find aggregated mean 
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


In [11]:
# do the same aggregate function above for the subset for comparison
df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


### The aggregate calculation for the whole dataset contains all of the department id's rather than a small slice.  Departments that were included in the slice have slightly different aggregate means in the whole dataframe output because additional columns are taken into consideration that were excluded from the sliced dataframe

## 04 Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.


In [12]:
# Create the max order column for each user id
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [13]:
# Create the sub-categories using the max order column 
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular Customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [14]:
# Check output
ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,department_id,prices,_merge,price_range_loc,busiest_day,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,7,9.0,both,Mid-range product,Regularly busy,Mid-range product,Regularly busy,Fewest orders,10,New customer
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Mid-range product,Least busy,Fewest orders,10,New customer
2,473747,1,prior,3,3,12,21.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Mid-range product,Least busy,Most orders,10,New customer
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,7,9.0,both,Mid-range product,Least busy,Mid-range product,Least busy,Fewest orders,10,New customer
4,431534,1,prior,5,4,15,28.0,196,1,1,...,7,9.0,both,Mid-range product,Least busy,Mid-range product,Least busy,Most orders,10,New customer


## 05 The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [15]:
# Group dataframe by loyalty group
# For each loyalty group, calculate summary statistics (mean, min, max)
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular Customer  12.495717  1.0  99999.0

## 06 The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:

## If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.” 
## If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [16]:
# Create a column to find the average price of purchases grouped by user id 
ords_prods_merge['avg_order_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [17]:
# Create the sub-categories using the avg order price column 
ords_prods_merge.loc[ords_prods_merge['avg_order_price'] >= 10, 'spending_flag'] = 'High Spender'
ords_prods_merge.loc[ords_prods_merge['avg_order_price'] < 10, 'spending_flag'] = 'Low Spender'

In [23]:
ords_prods_merge['spending_flag'].value_counts(dropna = False)

Low Spender     31770614
High Spender      634245
Name: spending_flag, dtype: int64

In [18]:
# Check output 
ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,_merge,price_range_loc,busiest_day,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_order_price,spending_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,both,Mid-range product,Regularly busy,Mid-range product,Regularly busy,Fewest orders,10,New customer,6.367797,Low Spender
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,both,Mid-range product,Regularly busy,Mid-range product,Least busy,Fewest orders,10,New customer,6.367797,Low Spender
2,473747,1,prior,3,3,12,21.0,196,1,1,...,both,Mid-range product,Regularly busy,Mid-range product,Least busy,Most orders,10,New customer,6.367797,Low Spender
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,both,Mid-range product,Least busy,Mid-range product,Least busy,Fewest orders,10,New customer,6.367797,Low Spender
4,431534,1,prior,5,4,15,28.0,196,1,1,...,both,Mid-range product,Least busy,Mid-range product,Least busy,Most orders,10,New customer,6.367797,Low Spender


## 07 In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:

## If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”

## If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”

## If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [19]:
# Creating a column to find the median days since prior order for each user id 
ords_prods_merge['median_days_since_prior_order'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [20]:
# Create the sub-categories using the median days since prior order column created above
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] > 20, 'frequency_tag'] = 'Non-frequent Customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_since_prior_order'] > 10) & (ords_prods_merge['median_days_since_prior_order'] <= 20), 'frequency_tag'] = 'Regular Customer'
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] <= 10, 'frequency_tag'] = 'Frequent Customer'

In [21]:
# Check output 
ords_prods_merge['frequency_tag'].value_counts(dropna = False)

Frequent Customer        21559853
Regular Customer          7208564
Non-frequent Customer     3636437
NaN                             5
Name: frequency_tag, dtype: int64

## 08 Exporting dataframe 

In [22]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data ', 'Prepared Data ', 'ords_prods_merge_updated2.pkl'))

OSError: Cannot save file into a non-existent directory: '/Users/brookecoffey/10-07-2023 Instacart Basket Analysis/02 Data /Prepared Data '